# LangChain Expression Language (LCEL)

LangChain Expression Language (LCEL)，是 Langchain 自定义的一种表达式语言，主要用于各种 “链”(chain) 的构建和组合，通过 LCEL 定义链，表达简洁清晰，易于理解。

LCEL 是这样实现的：

1. 定义了基础抽象，即 `Runnable` 接口类型，LCEL 定义的链，每个组成部分，本质都是 `Runnable` 类型的对象
2. 定义了构建链的基础规则：“顺序链”和“并行链”，分别用于 Runnable 对象的顺序执行和并行执行，其定义的类，分别为 `RunnableSequence` 和 `RunnableParallel`，其他形态的链或链的组成部件，都是这两个基础规则的衍生或组合
3. Langchain 通过运算符重载，重载了管道符（`|`），并通过隐式的强制转换，让 python 的函数(function) 和字典(dict) 自动转换为 Runnable 类型的对象，让链的编排更加简洁




## LCEL Get Started

为了演示 LCEL 的使用，我们首先要有几个 Runnable 类型的对象，为便于理解，我们通过 Langchain 的 `RunnableLambda` 定义几个简单的 Runnable 对象出来




In [25]:
from langchain_core.runnables import RunnableLambda

def some_func_x(x):
    return 2*x

def some_func_y(y):
    return 4*y

runnable1 = RunnableLambda(some_func_x) # 把输入翻倍
runnable2 = RunnableLambda(some_func_y) # 把输入翻4倍
runnable3 = RunnableLambda(lambda x: x) # 原封不动返回输入

print(runnable1.invoke("x"))
print(runnable2.invoke("y"))
print(runnable3.invoke("z"))

xx
yyyy
z


### RunnableSequence

RunnableSequence，即顺序链，就是把一组 Runnable 对象从左往右串起来一次执行，第一个的输出，作为第二个的结果，像我们之前用过的 `prompt | llm | parser`，就是顺序链，其本质是：

```python
chain = RunnableSequence([runnable1, runnable2])
```

当我们定义如下一个 `chain`，并执行 `chain.invoke("x")` 时，具体执行逻辑为：

```python
chain = runnable1 | runnable2 | runnable3
chain.invoke("x")

# 上述代码等价于：
output1 = runnable1.invoke("x")
output2 = runnable2.invoke(output1)
final_output = runnable3.invoke(output2)
```



In [33]:
from langchain_core.runnables import RunnableSequence
chain = runnable1 | runnable2
print(chain)
print(chain.invoke("xy,"))

chain2 = RunnableSequence(runnable1, runnable2)
assert chain2 == chain
print(chain2.invoke("xy,"))

first=RunnableLambda(some_func_x) middle=[] last=RunnableLambda(some_func_y)
xy,xy,xy,xy,xy,xy,xy,xy,
xy,xy,xy,xy,xy,xy,xy,xy,


### RunnableParallel

RunnableParallel 会把参数传递给 dict 中的每一组，其返回结果是一个 dict，该 dict 的每一组 (key, value)，分别为一个chain 的输出


In [36]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({
    "first": runnable1,
    "second": runnable2,
})

res = chain.invoke("abc,")
assert type(res) == dict
print(res)

{'first': 'abc,abc,', 'second': 'abc,abc,abc,abc,'}


## Runnable 的组合及强制转换

Runnable 可以随意组合，强制转换，主要是 python 的 dict ，可以隐式转换为 RunnableParallel，python 的 function，可以隐式转换为 RunnableLambda

值得注意的是，强制转换的条件是，dict 或 function 要在管道符的一侧，而另一侧必须是一个 Runnable 对象。


In [55]:
def func1(x):
    return x

chain = func1 | runnable1 | {"first": runnable2, "second": runnable3}
res = chain.invoke("abc,")
assert type(res) == dict
print(res)

{'first': 'abc,abc,abc,abc,abc,abc,abc,abc,', 'second': 'abc,abc,'}


In [56]:
chain =  {"first": runnable1, "second": runnable2} | runnable3 | (lambda x: x)
res = chain.invoke("abc,")
print(res)

{'first': 'abc,abc,', 'second': 'abc,abc,abc,abc,'}


# RunnableLambda

In [23]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

def some_func_x(x):
    return x

def some_func_y(y):
    return 2*y

def some_func_z(z):
    return z

runnable_func_x = RunnableLambda(some_func_x)
print(runnable_func_x.invoke("x"))
# print(runnable_func_x.invoke({"x": "x"}))
# print(runnable_func_x.invoke({"x"}))
# print(runnable_func_x.invoke(input="x"))


chain = RunnableLambda(some_func_x) | RunnableLambda(some_func_y)
print(chain.invoke("x"))
chain = RunnableLambda(some_func_x) | some_func_y
print(chain.invoke("x"))
chain = some_func_x | RunnableLambda(some_func_y)
print(chain.invoke("x"))

chain = RunnableParallel({"x": RunnableLambda(some_func_x), "y": RunnableLambda(some_func_y)})
print(chain.invoke("x"))

chain = RunnableParallel({"x": RunnableLambda(some_func_x), "y": RunnableLambda(some_func_y)}) | RunnableLambda(some_func_z)
print(chain.invoke("x"))

chain = {"x": RunnableLambda(some_func_x), "y": RunnableLambda(some_func_y)} | RunnableLambda(some_func_z)
print(chain.invoke("x"))

chain = {"x": some_func_x, "y": some_func_y} | RunnableLambda(some_func_z)
print(chain.invoke("x"))

x
xx
xx
xx
{'x': 'x', 'y': 'xx'}
{'x': 'x', 'y': 'xx'}
{'x': 'x', 'y': 'xx'}
{'x': 'x', 'y': 'xx'}
